# Skin cancer prediction

## Information

This notebook might be used as a test of the best model of our CI project.
More information along with an android app (apk) can be obtained:

https://skin-moles.web.app/

This can be used freely for any purpose.

## Prediction

In [6]:
%pip install tensorflow
%pip install numpy
%pip install matplotlib
%pip install scikit-image

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


"c:\program" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"c:\program" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"c:\program" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"c:\program" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [7]:
import tensorflow as tf
import matplotlib.pyplot as plt

### Setup Input Pipeline

Download

*   Scratch Model.
*   Test images.



In [8]:
! wget https://github.com/blobquiet/skin-moles-site/raw/main/model_all.h5
! wget https://github.com/blobquiet/skin-moles-site/raw/main/data_test.zip
! unzip data_test.zip >/dev/null

"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
El sistema no puede encontrar la ruta especificada.


### Load the images

In [9]:
import os
from PIL import Image
import numpy as np
from skimage.transform import resize
import time
ti = time.time()

folder_benign_test = './data_test/benign'
folder_malignant_test = './data_test/malignant'

read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))

# Load in testing pictures
ims_benign = [read(os.path.join(folder_benign_test, filename)) for filename in os.listdir(folder_benign_test)]
X_benign_test = np.array(ims_benign, dtype='uint8')
ims_malignant = [read(os.path.join(folder_malignant_test, filename)) for filename in os.listdir(folder_malignant_test)]
X_malignant_test = np.array(ims_malignant, dtype='uint8')

# Create labels
y_benign_test = np.zeros(X_benign_test.shape[0])
y_malignant_test = np.ones(X_malignant_test.shape[0])

# Merge data 
X_test = np.concatenate((X_benign_test, X_malignant_test), axis = 0)
y_test = np.concatenate((y_benign_test, y_malignant_test), axis = 0)

# Shuffle data
s = np.arange(X_test.shape[0])
np.random.shuffle(s)
X_test = X_test[s]
y_test = y_test[s]

# get val data
X_val = X_test[:100]

# convert data type and normalize values
X_test = X_test.astype('float32')
X_test /= 255

# 5x5 visualization 
class_names = ['Benign', 'Malignant']
plt.figure(figsize=(10,10))

for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_test[i], cmap=plt.cm.binary)
    plt.title(class_names[y_test[i].astype(int)])
plt.show()

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: './data_test/benign'

### Load the pre-trained model

In [ ]:
model = tf.keras.models.load_model('model_all.h5')
model.summary()

### Prediction of the model

In [ ]:
# 5x5 visualization 
class_names = ['Benign', 'Malignant']
plt.figure(figsize=(20,20))

pred = (model.predict(X_val) > 0.5).astype("int32")
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_val[i], cmap=plt.cm.binary)
    img = resize(X_val[i],(1,224,224,3))
    if  pred[i] == 1:
      prediction = class_names[1]
    elif pred[i] == 0:
      prediction = class_names[0]
    plt.title("Label: "+class_names[y_test[i].astype(int)] + "\nPrediction:"+prediction)
plt.show()